# Notebook 2: Apprentissage automatique, Regression: Polution CO2

#### Importation des librairies

In [232]:
import pandas as pd
import numpy as np
#-------------------------------------------------
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn.metrics import *
#-------------------------------------------------
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline

from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.compose import TransformedTargetRegressor, make_column_transformer

#--------------------------------------------------------
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression, SGDRegressor

#--------------------------------------------------------
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.inspection import permutation_importance
#------------------------------------------------
from xgboost import XGBRegressor

#-----------------------------------------------------------
from sklearn.model_selection import GridSearchCV

## Checklist

Depuis 2001, **l’ADEME** acquiert tous les ans ces données auprès de **l’Union Technique de l’Automobile du motocycle et du Cycle UTAC** (en charge de l’homologation des véhicules avant leur mise en vente) en accord avec le ministère du développement durable.
Pour chaque véhicule les données d’origine (transmises par l’Utac) sont les suivantes :

* **Les consommations de carburant**

* **Les émissions de dioxyde de carbone (CO2)**

* **Les émissions des polluants de l’air** (réglementés dans le cadre de la norme Euro)

* **L’ensemble des caractéristiques techniques des véhicules** (gammes, marques, modèles, n° de CNIT, type d’énergie ...)



## L'inventaire des varaibles pertinentes:

Les données comprenent des variables pertinentes suivantes:

* **lib_mrq_utac**: La marque, il y'a 12.

* **lib_mod**: Le modèle commerciale, il y'a 20.

* **cod_cbr**: Le type de carburant, il y a 5.

* **hybride**: Information permettant d’identifier les véhicules hybrides (O/N)

* **puiss_max** : Puissance maximale

* **typ_boite_nb_rapp**: Type boite de vitesse et le nombre de rapport.

* **conso_urb**: Consommation urbaine de carburant (en l/100km),

* **conso_exurb**: consommation extra urbaine de carburant (en l/100km),

* **conso_mixte**: Consommation mixte de carburant (en l/100km),

* **co2**: Emission de CO2 (en g/km),

* **masse_ordma_min**: Masse en ordre de marche mini

* **masse_ordma_max**: Masse en ordre de marche max
 
* **Carrosserie**: Carrosserie

* **gamme**: Gamme du véhicule



## Objectif

Notre objectif majeur dans ce notebook 2 est de :

Prédire les emisisions de **CO2** des vehicules en fonction de certaines informations (Variables explicatives)

* En utilisant 4 à 5 modéles différents

* En comparant  les scores
    
* En choissisant le meilleur modèle


## Description des données

Lien vers les données: https://www.data.gouv.fr/fr/datasets/emissions-de-co2-et-de-polluants-des-vehicules-commercialises-en-france/


# Chargement, lecture, apercu et infos des données

In [233]:
data_model = pd.read_csv("./data/data_model.csv")

In [234]:
unique_values = data_model['Carrosserie'].unique()
print(" : ",unique_values)

 :  ['BERLINE' 'BREAK' 'COUPE' 'CABRIOLET' 'TS TERRAINS/CHEMINS' 'COMBISPACE'
 'MINISPACE' 'MONOSPACE COMPACT' 'MONOSPACE' 'MINIBUS' 'COMBISPCACE']


### Selectionner les feautures les plus importans

In [235]:
df = data_model[['Carrosserie', 'masse_ordma_min', 'masse_ordma_max', 'co2']]

In [236]:
from sklearn.preprocessing import LabelEncoder
le= LabelEncoder()
objet_columns = df.select_dtypes(include='object').columns
for element in objet_columns:
    df.loc[:, element]=le.fit_transform(df.loc[:,element])

In [237]:
mapping_carrosserie= dict(zip(df['Carrosserie'], data_model['Carrosserie']))

### Traiter la colonne Carrosserie

In [238]:
df.head(10)

,Carrosserie,masse_ordma_min,masse_ordma_max,co2
0,0,1505.0,1505.0,182.0
1,0,1555.0,1555.0,186.0
2,0,1565.0,1565.0,134.0
3,0,1565.0,1565.0,134.0
4,0,1565.0,1565.0,139.0
5,0,1565.0,1565.0,139.0
6,0,1565.0,1565.0,136.0
7,1,1615.0,1615.0,137.0
8,1,1615.0,1615.0,137.0
9,1,1615.0,1615.0,142.0


In [239]:
data_model.head(10)

,lib_mrq,cnit,cod_cbr,hybride,puiss_max,Carrosserie,gamme,co2,masse_ordma_min,masse_ordma_max,Type_boite,Nb_rapp
0,ALFA-ROMEO,M10ALFVP000G340,ES,non,147,BERLINE,MOY-SUPER,182.0,1505.0,1505.0,M,6.0
1,ALFA-ROMEO,M10ALFVP000H341,ES,non,147,BERLINE,MOY-SUPER,186.0,1555.0,1555.0,M,6.0
2,ALFA-ROMEO,M10ALFVP000E302,GO,non,100,BERLINE,MOY-SUPER,134.0,1565.0,1565.0,M,6.0
3,ALFA-ROMEO,M10ALFVP000F303,GO,non,100,BERLINE,MOY-SUPER,134.0,1565.0,1565.0,M,6.0
4,ALFA-ROMEO,M10ALFVP000G304,GO,non,125,BERLINE,MOY-SUPER,139.0,1565.0,1565.0,M,6.0
5,ALFA-ROMEO,M10ALFVP000H305,GO,non,125,BERLINE,MOY-SUPER,139.0,1565.0,1565.0,M,6.0
6,ALFA-ROMEO,M10ALFVP000U221,GO,non,125,BERLINE,MOY-SUPER,136.0,1565.0,1565.0,M,6.0
7,ALFA-ROMEO,M10ALFVP000J306,GO,non,100,BREAK,MOY-SUPER,137.0,1615.0,1615.0,M,6.0
8,ALFA-ROMEO,M10ALFVP000J307,GO,non,100,BREAK,MOY-SUPER,137.0,1615.0,1615.0,M,6.0
9,ALFA-ROMEO,M10ALFVP000K308,GO,non,125,BREAK,MOY-SUPER,142.0,1615.0,1615.0,M,6.0


### preparation de la dataframe au classifiers
    
Pour chacun de nos modèles:
   * **DummyRegressor**,
   * **LinearRegression**, 
   * **SGDRegressor**, 
   * **RandomForestRegressor**,
   * **GradientBoostingRegressor**,
   * **XGBRegressor**, 
    
Nous allons predire le niveau d'émission de **Co2** puis établir une comparaison entre les performances de chaque modèle.

    Appliquer un GridSearch pour optimiser les hyperparamètres de chaque modèle
    Paramètrer dans GridSearch les scores qui sont adaptés à la régression (R2, MAE et RMSE)
    Identifier le meilleur modèle

In [240]:
X = df.drop(['co2'] ,axis =1)
y = df['co2']


scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)



In [241]:
# Create the label encoder
le = LabelEncoder()

# Fit the label encoder to the target variable
le.fit(y)

# Transform the target variable
y= le.transform(y)

# IA stuf

## liste des  classifiers et des param_grids

In [242]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV, KFold

classifiers = [DummyRegressor(),LinearRegression(),SGDRegressor(),RandomForestRegressor(),GradientBoostingRegressor(),XGBRegressor()]
param_grids = [
    {'strategy': ['mean', 'median'], 'constant': [0, 1]},
    {'fit_intercept': [True, False]},
    {'alpha': [0.0001, 0.001], 'penalty': ['l1', 'l2']},
    {'n_estimators': [50, 100, 200], 'max_depth': [None, 2, 4], 'min_samples_split': [3, 5, 7]},
    {'n_estimators': [50, 100], 'learning_rate': [0.1, 0.01], 'max_depth': [3, 5]},
    {'learning_rate': [0.1, 0.01], 'max_depth': [3, 5], 'n_estimators': [50, 80]}]

## TEST des classifiers-param_grids

In [243]:


results = []

for i, (clf, param_grid) in enumerate(zip(classifiers, param_grids)):
    # Create the GridSearchCV object
    grid_search = GridSearchCV(clf, param_grid, scoring='r2', cv=5)
    # Fit the GridSearchCV object to the data
    grid_search.fit(X, y)
    # Store the results
    results.append((type(clf).__name__, grid_search.best_params_, grid_search.best_score_))

    print("=============================================================")
    print(f"regreseur {i+1}: {type(clf).__name__}")
    print(f"parameters: {grid_search.best_params_}")
    print(f"score: {grid_search.best_score_}")

winner = max(results, key=lambda x: x[2])

print("=============================================================")
print(f"The winner is {winner[0]} with a score of {winner[2]} and parameters:")
print(winner[1])
print("=============================================================")

regreseur 1: DummyRegressor
parameters: {'constant': 0, 'strategy': 'median'}
score: -0.23304448159725616
regreseur 2: LinearRegression
parameters: {'fit_intercept': True}
score: 0.16237163660500403


regreseur 3: SGDRegressor
parameters: {'alpha': 0.0001, 'penalty': 'l2'}
score: 0.17363905201033125
regreseur 4: RandomForestRegressor
parameters: {'max_depth': None, 'min_samples_split': 3, 'n_estimators': 50}
score: 0.29393680906899866
regreseur 5: GradientBoostingRegressor
parameters: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 50}
score: 0.374150685596158
regreseur 6: XGBRegressor
parameters: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 50}
score: 0.36923339113021464
The winner is GradientBoostingRegressor with a score of 0.374150685596158 and parameters:
{'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 50}


# extraction du model

Développer une application Streamlit avec les options suivantes :

    1- L'utilisateur doit selectionner le type du carrosserie à partire d'un menu déroulant
    2- L'utilisateur doit saisir 'masse_ordma_min' et 'masse_ordma_max' dans deux champs de saisie différent
    3- Programmer un boutton pour lancer la prédiction de CO2

## creation d'un model avec le meilleur resultat classifiers-param_grids
    masse_ordma_min = 825.0 - 2760.0
    masse_ordma_max = 825.0 - 3094.0
    Carrosserie = 0 - 10

    co2 = 13.0 - 572.0

In [244]:

best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_

best_estimator.set_params(**best_params)
model = best_estimator
#model = RandomForestRegressor(max_depth= None, min_samples_split= 7, n_estimators=100)

model.fit(X, y)
print("R2 score: {:.4f}".format(model.score(X, y)))




R2 score: 0.6855


## exportation du model , scaler et d'une bublioteque qui map la colonne "Carrosserie"

In [245]:
import joblib
joblib.dump(model,'./pkl_data/model.pkl')
joblib.dump(mapping_carrosserie,'./pkl_data/maping.pkl')
joblib.dump(scaler,'./pkl_data/scaler.pkl')


['./pkl_data/scaler.pkl']

# Aplication web


suite a se notebook  le model est pret a l'utilisation.
utilisé la commande 

In [ ]:
"""


streamlit  run app.py


"""